In [1]:
import pandas as pd

In [39]:
election_year = 2021
election_round = "first_round"
country = "Chile"

df = pd.read_csv(f"data_output/{country}/{election_year}_{election_round}.csv.gz")
df.head()

print(df.shape)
print(f"Nro. candidates: {len(df.candidate.unique())}")
print(df.candidate.unique())


(466390, 6)
Nro. candidates: 10
['EDUARDO ARTES BRICHETTI' 'FRANCO PARISI FERNANDEZ' 'GABRIEL BORIC FONT'
 'JOSE ANTONIO KAST RIST' 'MARCO ENRIQUEZ-OMINAMI GUMUCIO'
 'SEBASTIAN SICHEL RAMIREZ' 'YASNA PROVOSTE CAMPILLAY' 'ABSTENTION' nan
 'BLANK']


array(['FRANCO ALDO PARISI FERNANDEZ', 'MARCEL CLAUDE REYES',
       'RICARDO ISRAEL ZIPPER', 'MARCO ENRIQUEZ-OMINAMI GUMUCIO',
       'ROXANA MIRANDA MENESES', 'MICHELLE BACHELET JERIA',
       'EVELYN MATTHEI FORNET', 'ALFREDO SFEIR YOUNIS',
       'TOMAS JOCELYN-HOLT LETELIER'], dtype=object)